# 학습한 모델로 예측하기

In [8]:
!nvidia-smi

Tue Jun  7 10:59:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.142.00   Driver Version: 450.142.00   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   31C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Library

In [3]:
import os
import cv2
import time
import random
import logging  # 로그 출력
import easydict  # 속성으로 dict 값에 access할 수 있음
import numpy as np
import pandas as pd
from tqdm import tqdm  # process bar
from os.path import join as opj
from ptflops import get_model_complexity_info
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

import timm
import torch
import torch.nn as nn
import torch_optimizer as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, grad_scaler
from torchvision import transforms

import warnings
warnings.filterwarnings('ignore')

# Config

In [4]:
args = easydict.EasyDict(
    {'exp_num':'0',
     
     # Path settings
     # /home/lab18/Data/product_image/Training/image/10268_아넬라사과디저트_2입/
     'data_path':'/home/lab16/jupyter_home/Data/product_image/',
     
     # Test dataset
#      'test_image_path':'/home/lab16/jupyter_home/Data/product_image/test/',
     'test_image_path':'/home/lab16/jupyter_home/Data/product_image/Validation/total_image/',
     'Kfold':1,
     'model_path':'results/',

     # Model parameter settings 
     'encoder_name':'regnety_064',
     'drop_path_rate':0.2,
     
     # Training parameter settings
     ## Base Parameter
     'img_size':224,
     'batch_size':16,
     'epochs':50,
     'optimizer':'Lamb',
#      'optimizer':'Adadelta',
     'initial_lr':5e-6,
     'weight_decay':1e-3,

     ## Augmentation
     'aug_ver':2,

     ## Scheduler (OnecycleLR)
     'scheduler':'cycle',
     'warm_epoch':5,
     'max_lr':1e-3,

     ### Cosine Annealing
     'min_lr':5e-6,
     'tmax':145,

     ## etc.
     'patience':10,
     'clipping':None,

     # Hardware settings
     'amp':True,
     'multi_gpu':False,
     'logging':False,
     'num_workers':0,
     'seed':42
    })

# Dataset & Loader

In [16]:
class Test_dataset(Dataset):
    def __init__(self, path, transform=None):
#         total_images_path = glob(args.test_image_path + '*.jpg')
        total_images_path = glob(path + '*.jpg')
        file_names = []
        for i in range(len(total_images_path)):
            file_names.append(os.path.basename(total_images_path[i]))
            file_names.sort()
        file_names = np.array(file_names)

        self.test_file_name = file_names
        self.transform = transform

        print(f'Test Dataset size:{len(self.test_file_name)}')
        print(self.test_file_name)

    def __getitem__(self, idx): # test 경로에 있는 png 이미지 읽어서 float32로 변환
#         image = cv2.imread(opj('./data/test_256_new/', self.test_file_name[idx])).astype(np.float32)
#         image = cv2.imread(opj(args.test_image_path, self.test_file_name[idx])).astype(np.float32)
        image = cv2.imread(opj(path, self.test_file_name[idx])).astype(np.float32)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) / 255.0  # BGR=>RGB 변환

        if self.transform is not None:
            image = self.transform(torch.from_numpy(image.transpose(2,0,1)))

        return image

    def __len__(self):
        return len(self.test_file_name)

# def get_loader(df, phase: str, batch_size, shuffle, num_workers, transform):
def get_loader(phase: str, batch_size, shuffle, num_workers, transform):
    if phase == 'test':
#         dataset = Test_dataset(df, transform)  
        dataset = Test_dataset(transform) 
        data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, pin_memory=True)
        
    # 임시로 추가
    elif phase == 'validation':
#         dataset = Train_Dataset(df, transform)
        dataset = Valid_Dataset(transform)
        data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, pin_memory=True)
        
    else:
#         dataset = Train_Dataset(df, transform)
        path = ''
        dataset = Train_Dataset(transform)
        data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, pin_memory=True,
                                 drop_last=False)
        
    return data_loader

def get_train_augmentation(img_size, ver):
    if ver == 1: # for validset
        transform = transforms.Compose([
                transforms.Resize((img_size, img_size)),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
#                 transforms.ToTensor()
                ])

    if ver == 2:
        transform = transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomCrop(224),
                transforms.RandomPerspective(),
                transforms.RandomAffine((20)),  # x, y축으로 이미지 늘림
                transforms.RandomRotation(90),
                transforms.Resize((img_size, img_size)),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
#                 transforms.ToTensor()
            ])
    
    return transform

# Network

In [17]:
class Network(nn.Module):
    def __init__(self, args):
        super().__init__()
        # 사전 학습된 모델 사용하기
        self.encoder = timm.create_model(args.encoder_name, pretrained=True,
                                    drop_path_rate=args.drop_path_rate,
                                    )
        
        if 'regnet' in args.encoder_name:        
            num_head = self.encoder.head.fc.in_features
            self.encoder.head.fc = nn.Linear(num_head, 76)
        
        elif 'efficient' in args.encoder_name:
            num_head = self.encoder.classifier.in_features
            self.encoder.classifier = nn.Linear(num_head, 76)

    def forward(self, x):
        x = self.encoder(x)
        return x

class Network_test(nn.Module):
    def __init__(self, encoder_name):
        super().__init__()
        self.encoder = timm.create_model(encoder_name, pretrained=True,
                                    drop_path_rate=0,
                                    )
        
        if 'regnet' in encoder_name:        
            num_head = self.encoder.head.fc.in_features
            self.encoder.head.fc = nn.Linear(num_head, 76)
        
        elif 'efficient' in encoder_name:
            num_head = self.encoder.classifier.in_features
            self.encoder.classifier = nn.Linear(num_head, 76)
    
    def forward(self, x):
        x = self.encoder(x)
        return x

# Inference

In [18]:
def predict(encoder_name, test_loader, device, model_path):
    model = Network_test(encoder_name).to(device)
    model.load_state_dict(torch.load(opj(model_path, 'best_model.pth'))['state_dict'])
    model.eval()
    preds_list = []
    with torch.no_grad():
        for images in tqdm(test_loader):
            images = torch.as_tensor(images, device=device, dtype=torch.float32)
            preds = model(images)
            preds = torch.softmax(preds, dim=1)
            preds_list.extend(preds.cpu().tolist())

    return np.array(preds_list)

# def ensemble_5fold(model_path_list, test_loader, device):
#     predict_list = []
#     for model_path in model_path_list:
#         prediction = predict(encoder_name= 'regnety_064', 
#                              test_loader = test_loader, device = device, model_path = model_path)
#         predict_list.append(prediction)
#     ensemble = (predict_list[0] + predict_list[1] + predict_list[2] + predict_list[3] + predict_list[4])/len(predict_list)

#     return ensemble

In [19]:
# fold X
def result(model_path_list, test_loader, device):
    predict_list = []
    for model_path in model_path_list:
        prediction = predict(encoder_name= 'regnety_064', 
                             test_loader = test_loader, device = device, model_path = model_path)
        predict_list.append(prediction)
    ensemble = predict_list[0]

    return ensemble

# 모델 불러오기

In [20]:
models_path=[]

In [21]:
save_path = os.path.join(args.model_path, (str(0)).zfill(3))
models_path.append(save_path)

In [22]:
models_path

['results/000']

# 예측하기

In [23]:
img_size = 224

In [40]:
from glob import glob

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [41]:
path = '/home/lab16/jupyter_home/Data/product_image/Validation/total_image/'
# path = '/home/lab16/jupyter_home/Data/product_image/test/'

In [42]:
test_transform = get_train_augmentation(img_size=img_size, ver=1)
test_dataset = Test_dataset(path, test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

Test Dataset size:1140
['10242_00_m_19.jpg' '10242_00_m_4.jpg' '10242_00_s_12.jpg' ...
 '70208_60_s_15.jpg' '70208_60_s_2.jpg' '70208_60_s_7.jpg']


In [43]:
predict_arr = result(models_path, test_loader, device)

100%|███████████████████████████████████████████| 18/18 [03:10<00:00, 10.60s/it]


In [44]:
predict_arr

array([[7.83319771e-02, 2.53187463e-05, 2.95737518e-05, ...,
        8.24016533e-05, 5.76455079e-07, 2.79452622e-01],
       [1.03305176e-01, 4.67418577e-05, 1.07280073e-04, ...,
        1.38688862e-04, 9.20826153e-07, 1.69329613e-01],
       [3.16903204e-01, 3.55710131e-06, 7.79306629e-06, ...,
        1.88238613e-04, 3.96674409e-07, 4.07990456e-01],
       ...,
       [1.75382520e-04, 2.10436519e-06, 3.21743050e-06, ...,
        4.53612083e-05, 1.05027630e-06, 9.34458733e-01],
       [1.35293012e-04, 3.67844495e-06, 3.37169899e-06, ...,
        2.55744089e-05, 6.61869080e-07, 9.34741020e-01],
       [1.12702975e-04, 8.94627021e-07, 1.74231479e-06, ...,
        2.01924686e-05, 4.21799058e-07, 9.73867714e-01]])

In [53]:
# predict_arr.argmax(axis=1) # array([75, 75, 13, 75, 75, 75, 75, 75, 75, 75, 75, 13])

In [45]:
labels = ['삼양사)건포도150G', '쁘띠첼요거젤리밀감', '매일데르뜨복숭아3개입90G_3', '매일데르뜨자몽130G', '신선에프앤브이)파인애플컵400G', '씨제이)쁘티첼(요거젤리블루베리)', '팜팩토리)아넬라사과_자두디저트', 'CJ쟈뎅)쁘띠첼과일젤리포도90G', 'CJ쟈뎅)쁘띠첼과일젤리복숭아90G', '돌코리아애플팝496ML', '풍림푸드피코크포도젤리90G', '엠디에스코리아)사과푸딩220G', '롯데제과)디저뜨와(구운치즈케이크타르트)', '매일유업)데르뜨130G', '씨제이)쁘티첼(요거젤리딸기)', '신선에프앤브이)파인애플컵100G', '홈플러스)마이프루타애플앤망고100G', '홈플러스)마이프루타애플앤블루베리100G', '팜팩토리)아넬라사과_바나나디저트', '대만)망고케익184g', '홈플러스)마이프루타애플앤페어100G', 'CJ쟈뎅)쁘띠첼과일젤리밀감270G', '홈플러스)마이프루타애플앤스트로베리200G', '홈플러스)마이프루타애플앤블루베리200G', '풍림푸드망고젤리4개입90G_4', '매일데르뜨파인애플90G', '세방유통)애플망고슬라이스인망고쥬스370G', '세방유통)골든파인애플슬라이스인시럽370G', '금광약초)대추', '팜팩토리)하넬라사과_망고디저트', '돌트로피칼666G', '이멕스무역)혼합푸딩(ASSORTEDPUDDING)4개입', 'CJ쟈뎅)쁘띠첼과일젤리복숭아270G', '팜팩토리)아넬라배_사과디저트', '매일데르뜨자몽3개입130G_3', '씨제이)쁘티첼(요거젤리화이트코코)', '호남샤니)제리뽀사과맛', '영도물산)통밀도넛츠', '풍림푸드포도젤리4개입90G_4', '매일데르뜨감귤3개입90G_3', '호남샤니)카페메이트초코블랑', '홈플러스)마이프루타애플앤스트로베리100G', 'CJ쟈뎅)쁘띠첼과일젤리밀감90G', '풍림푸드피코크망고젤리90G', '돌코리아백도젤리90G', '호남샤니)제리뽀', '풍림푸드복숭아젤리4개입90G_4', '돌코리아망고컵198G', '오뚜기스위트앤젤밀감3개입90G_4', '홈플러스)마이프루타애플앤망고200G', '아넬라사과디저트_2입', '아이배냇)쌩마멧유기농사과', '매일데르뜨감귤90G', 'Dole후룻볼파인애플4입100_주스', '홈플러스)마이프루타애플앤페어200G', '씨제이)쁘티첼(요거젤리밀감)', '쁘띠첼요거젤리복숭아', '호남샤니)제리뽀딸기맛', '돌코리아코코파인젤리90G', '아이배냇)쌩마멧유기농사과_바나나', '호남샤니)제리뽀오렌지맛', '돌황도666G', '씨제이)쁘티첼(요거젤리복숭아)', '매일데르뜨백도3개입130G_3', '홈플러스)마이프루타애플앤바나나100G', '돌코리아)백도젤리', '홈플러스)마이프루타애플앤바나나200G', '호남샤니)제리뽀망고맛', 'Dole후룻볼슬라이스복숭아198g', '대만)파인애플케익184G', 'Dole후룻볼트로피칼4입100_주스', '아이배냇)쌩마멧유기농사과_배', '오뚜기스위트앤젤복숭아3개입90G_3', 'CJ쟈뎅)쁘띠첼과일젤리포도270G', '티디에프코리아)생미쉘갈라떼', '대만)파인애플케익250G']

In [46]:
labels = {string : i for i,string in enumerate(labels)}

In [47]:
label_decoder = {val:key for key, val in labels.items()}

In [48]:
pred_list = []

for i in range(len(test_dataset.test_file_name)):
    prediction = label_decoder[predict_arr.argmax(axis=1)[i]]
    print(prediction)
    pred_list.append(prediction)

씨제이)쁘티첼(요거젤리블루베리)
매일데르뜨자몽130G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
매일데르뜨자몽130G
매일데르뜨자몽130G
매일데르뜨자몽130G
매일데르뜨자몽130G
매일데르뜨자몽130G
씨제이)쁘티첼(요거젤리블루베리)
씨제이)쁘티첼(요거젤리블루베리)
씨제이)쁘티첼(요거젤리블루베리)
대만)파인애플케익250G
대만)파인애플케익250G
매일데르뜨자몽130G
씨제이)쁘티첼(요거젤리블루베리)
씨제이)쁘티첼(요거젤리블루베리)
씨제이)쁘티첼(요거젤리블루베리)
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
씨제이)쁘티첼(요거젤리블루베리)
매일데르뜨자몽130G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
매일데르뜨자몽130G
매일데르뜨자몽130G
매일데르뜨자몽130G
매일데르뜨자몽130G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
매일데르뜨자몽130G
매일데르뜨자몽130G
매일데르뜨자몽130G
매일데르뜨자몽130G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
매일데르뜨자몽130G
매일데르뜨자몽1

In [49]:
pred_list

['씨제이)쁘티첼(요거젤리블루베리)',
 '매일데르뜨자몽130G',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '매일데르뜨자몽130G',
 '매일데르뜨자몽130G',
 '매일데르뜨자몽130G',
 '매일데르뜨자몽130G',
 '매일데르뜨자몽130G',
 '씨제이)쁘티첼(요거젤리블루베리)',
 '씨제이)쁘티첼(요거젤리블루베리)',
 '씨제이)쁘티첼(요거젤리블루베리)',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '매일데르뜨자몽130G',
 '씨제이)쁘티첼(요거젤리블루베리)',
 '씨제이)쁘티첼(요거젤리블루베리)',
 '씨제이)쁘티첼(요거젤리블루베리)',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '씨제이)쁘티첼(요거젤리블루베리)',
 '매일데르뜨자몽130G',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '매일데르뜨자몽130G',
 '매일데르뜨자몽130G',
 '매일데르뜨자몽130G',
 '매일데르뜨자몽130G',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '대만)파인애플케익250G',
 '

In [50]:
import pandas as pd

In [51]:
pred_series = pd.Series(pred_list)

In [52]:
pred_series.describe()

count              1140
unique                7
top       대만)파인애플케익250G
freq                592
dtype: object

In [ ]:
# 25223대만)파인애플케익250G
# 35211매일유업)데르뜨130G

임의 데이터로 test

In [27]:
img_size = 224

In [28]:
from glob import glob

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [29]:
path = '/home/lab16/jupyter_home/Data/product_image/test/'

In [30]:
test_transform = get_train_augmentation(img_size=img_size, ver=1)
test_dataset = Test_dataset(path, test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

Test Dataset size:12
['001.jpg' '002.jpg' '003.jpg' '004.jpg' '005.jpg' '006.jpg' '007.jpg'
 '008.jpg' '009.jpg' '010.jpg' '011.jpg' '012.jpg']


In [31]:
predict_arr = result(models_path, test_loader, device)

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  4.41it/s]


In [32]:
predict_arr

array([[1.34814700e-05, 2.27700275e-05, 2.14073189e-05, 1.60843629e-04,
        7.37163077e-07, 1.89244957e-03, 7.28119742e-09, 2.81514862e-04,
        1.49561554e-08, 5.71856566e-04, 2.42128658e-06, 1.42948393e-05,
        6.41651043e-09, 9.95447934e-01, 1.54452719e-04, 3.75602463e-06,
        1.18988019e-08, 7.73696286e-07, 3.05876902e-09, 4.13687076e-06,
        2.32865371e-09, 4.02374326e-06, 1.97435490e-09, 8.35065875e-05,
        1.94119352e-08, 1.37193792e-05, 2.84114261e-08, 1.27933081e-05,
        1.33250091e-06, 1.50125402e-07, 2.05266453e-08, 1.11709895e-07,
        1.74629491e-07, 1.93930632e-08, 2.62410644e-08, 1.72198504e-08,
        4.33528825e-04, 2.57875781e-05, 3.83146289e-05, 8.28852875e-10,
        5.76304672e-12, 9.83085974e-11, 1.08052442e-10, 7.99886095e-07,
        4.26999902e-09, 1.16206165e-08, 2.24440044e-09, 2.58236899e-08,
        7.36216199e-10, 5.67010483e-09, 1.92209342e-07, 1.74149091e-06,
        1.11734098e-05, 1.01717924e-04, 4.67545797e-05, 2.081282

In [53]:
# predict_arr.argmax(axis=1) # array([75, 75, 13, 75, 75, 75, 75, 75, 75, 75, 75, 13])

In [33]:
labels = ['삼양사)건포도150G', '쁘띠첼요거젤리밀감', '매일데르뜨복숭아3개입90G_3', '매일데르뜨자몽130G', '신선에프앤브이)파인애플컵400G', '씨제이)쁘티첼(요거젤리블루베리)', '팜팩토리)아넬라사과_자두디저트', 'CJ쟈뎅)쁘띠첼과일젤리포도90G', 'CJ쟈뎅)쁘띠첼과일젤리복숭아90G', '돌코리아애플팝496ML', '풍림푸드피코크포도젤리90G', '엠디에스코리아)사과푸딩220G', '롯데제과)디저뜨와(구운치즈케이크타르트)', '매일유업)데르뜨130G', '씨제이)쁘티첼(요거젤리딸기)', '신선에프앤브이)파인애플컵100G', '홈플러스)마이프루타애플앤망고100G', '홈플러스)마이프루타애플앤블루베리100G', '팜팩토리)아넬라사과_바나나디저트', '대만)망고케익184g', '홈플러스)마이프루타애플앤페어100G', 'CJ쟈뎅)쁘띠첼과일젤리밀감270G', '홈플러스)마이프루타애플앤스트로베리200G', '홈플러스)마이프루타애플앤블루베리200G', '풍림푸드망고젤리4개입90G_4', '매일데르뜨파인애플90G', '세방유통)애플망고슬라이스인망고쥬스370G', '세방유통)골든파인애플슬라이스인시럽370G', '금광약초)대추', '팜팩토리)하넬라사과_망고디저트', '돌트로피칼666G', '이멕스무역)혼합푸딩(ASSORTEDPUDDING)4개입', 'CJ쟈뎅)쁘띠첼과일젤리복숭아270G', '팜팩토리)아넬라배_사과디저트', '매일데르뜨자몽3개입130G_3', '씨제이)쁘티첼(요거젤리화이트코코)', '호남샤니)제리뽀사과맛', '영도물산)통밀도넛츠', '풍림푸드포도젤리4개입90G_4', '매일데르뜨감귤3개입90G_3', '호남샤니)카페메이트초코블랑', '홈플러스)마이프루타애플앤스트로베리100G', 'CJ쟈뎅)쁘띠첼과일젤리밀감90G', '풍림푸드피코크망고젤리90G', '돌코리아백도젤리90G', '호남샤니)제리뽀', '풍림푸드복숭아젤리4개입90G_4', '돌코리아망고컵198G', '오뚜기스위트앤젤밀감3개입90G_4', '홈플러스)마이프루타애플앤망고200G', '아넬라사과디저트_2입', '아이배냇)쌩마멧유기농사과', '매일데르뜨감귤90G', 'Dole후룻볼파인애플4입100_주스', '홈플러스)마이프루타애플앤페어200G', '씨제이)쁘티첼(요거젤리밀감)', '쁘띠첼요거젤리복숭아', '호남샤니)제리뽀딸기맛', '돌코리아코코파인젤리90G', '아이배냇)쌩마멧유기농사과_바나나', '호남샤니)제리뽀오렌지맛', '돌황도666G', '씨제이)쁘티첼(요거젤리복숭아)', '매일데르뜨백도3개입130G_3', '홈플러스)마이프루타애플앤바나나100G', '돌코리아)백도젤리', '홈플러스)마이프루타애플앤바나나200G', '호남샤니)제리뽀망고맛', 'Dole후룻볼슬라이스복숭아198g', '대만)파인애플케익184G', 'Dole후룻볼트로피칼4입100_주스', '아이배냇)쌩마멧유기농사과_배', '오뚜기스위트앤젤복숭아3개입90G_3', 'CJ쟈뎅)쁘띠첼과일젤리포도270G', '티디에프코리아)생미쉘갈라떼', '대만)파인애플케익250G']

In [34]:
labels = {string : i for i,string in enumerate(labels)}

In [35]:
label_decoder = {val:key for key, val in labels.items()}

In [36]:
pred_list = []

for i in range(len(test_dataset.test_file_name)):
    prediction = label_decoder[predict_arr.argmax(axis=1)[i]]
    print(prediction)
    pred_list.append(prediction)

매일유업)데르뜨130G
대만)파인애플케익250G
대만)파인애플케익250G
매일유업)데르뜨130G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G


In [37]:
for i in range(len(test_dataset.test_file_name)):
    prediction = label_decoder[predict_arr.argmax(axis=1)[i]]
    print(prediction)

매일유업)데르뜨130G
대만)파인애플케익250G
대만)파인애플케익250G
매일유업)데르뜨130G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G
대만)파인애플케익250G


In [38]:
pred_series = pd.Series(pred_list)

In [39]:
pred_series.describe()

count                12
unique                2
top       대만)파인애플케익250G
freq                 10
dtype: object